# Lab 6: Scheduling and Timetables - Scheduling DAGs in Airflow

## 🎯 Objectives
- Understand how scheduling works in Airflow
- Use cron expressions for scheduling
- Use timedelta for interval-based scheduling
- Create custom timetables for complex scheduling
- Understand catchup and data intervals
- Handle timezone and scheduling edge cases

## 📋 Prerequisites
- Completed Lab 1-5
- Understand DAGs and tasks
- Airflow cluster is running

## 🏗️ Scheduling Overview
Airflow scheduling is based on:
- **Schedule Interval**: Time period between DAG runs
- **Start Date**: Start date of schedule
- **Catchup**: Whether to run missed runs
- **Data Intervals**: Time period for data processing
- **Timetables**: Custom logic for complex scheduling


## 1. Import Libraries and Setup


In [ ]:
# Import Airflow scheduling modules
from airflow.sdk import DAG, task
from airflow.providers.standard.operators.empty import EmptyOperator
from airflow.timetables.interval import CronDataIntervalTimetable
from airflow.timetables.base import Timetable
from airflow.timetables.trigger import CronTriggerTimetable

import pendulum
from datetime import datetime, timedelta

print("✅ Airflow scheduling modules imported successfully!")


## 2. Cron Expressions - Lập lịch với Cron

Cron expressions cho phép lập lịch linh hoạt với các preset và custom patterns.


In [ ]:
# DAG với Cron Presets
@dag(
    dag_id="cron_presets_example",
    schedule="@daily",  # Cron preset: Chạy hàng ngày lúc 00:00
    start_date=pendulum.datetime(2024, 1, 1, tz="UTC"),
    catchup=False,
    tags=["scheduling", "cron", "presets"],
)
def cron_presets_dag():
    """
    ### Cron Presets Example
    Sử dụng cron presets cho scheduling đơn giản.
    """
    
    @task
    def daily_task():
        """Task chạy hàng ngày"""
        print("Running daily task")
        return "Daily task completed"
    
    daily_task()

# Create DAG
cron_presets_dag_instance = cron_presets_dag()

print("✅ Cron Presets DAG created!")
print(f"Schedule: @daily (runs every day at 00:00)")
print("\n📋 Common Cron Presets:")
print("  - @once: Chạy một lần duy nhất")
print("  - @hourly: Chạy mỗi giờ")
print("  - @daily: Chạy hàng ngày lúc 00:00")
print("  - @weekly: Chạy hàng tuần (Chủ nhật)")
print("  - @monthly: Chạy hàng tháng (ngày 1)")
print("  - @yearly: Chạy hàng năm (1/1)")


## 3. Custom Cron Expressions

Cron expressions cho phép định nghĩa schedule patterns phức tạp hơn.


In [ ]:
# DAG với Custom Cron Expressions
@dag(
    dag_id="custom_cron_example",
    schedule="0 */6 * * *",  # Chạy mỗi 6 giờ (00:00, 06:00, 12:00, 18:00)
    start_date=pendulum.datetime(2024, 1, 1, tz="UTC"),
    catchup=False,
    tags=["scheduling", "cron", "custom"],
)
def custom_cron_dag():
    """
    ### Custom Cron Expression Example
    Sử dụng custom cron expression cho scheduling linh hoạt.
    """
    
    @task
    def scheduled_task():
        """Task chạy theo custom schedule"""
        print("Running scheduled task")
        return "Task completed"
    
    scheduled_task()

# Create DAG
custom_cron_dag_instance = custom_cron_dag()

print("✅ Custom Cron DAG created!")
print(f"Schedule: 0 */6 * * * (every 6 hours)")
print("\n📋 Cron Expression Format:")
print("  ┌───────────── minute (0-59)")
print("  │ ┌─────────── hour (0-23)")
print("  │ │ ┌───────── day of month (1-31)")
print("  │ │ │ ┌─────── month (1-12)")
print("  │ │ │ │ ┌───── day of week (0-6, Sunday=0)")
print("  │ │ │ │ │")
print("  * * * * *")
print("\n💡 Examples:")
print("  - '0 0 * * *': Daily at midnight")
print("  - '0 */3 * * *': Every 3 hours")
print("  - '0 9 * * 1-5': Weekdays at 9 AM")
print("  - '0 0 1 * *': First day of month")
print("  - '30 14 * * *': Daily at 2:30 PM")


In [ ]:
# DAG với Timedelta Scheduling
@dag(
    dag_id="timedelta_scheduling_example",
    schedule=timedelta(hours=2),  # Chạy mỗi 2 giờ
    start_date=pendulum.datetime(2024, 1, 1, tz="UTC"),
    catchup=False,
    tags=["scheduling", "timedelta"],
)
def timedelta_scheduling_dag():
    """
    ### Timedelta Scheduling Example
    Sử dụng timedelta cho interval-based scheduling.
    """
    
    @task
    def interval_task():
        """Task chạy theo interval"""
        print("Running interval-based task")
        return "Interval task completed"
    
    interval_task()

# Create DAG
timedelta_scheduling_dag_instance = timedelta_scheduling_dag()

print("✅ Timedelta Scheduling DAG created!")
print(f"Schedule: timedelta(hours=2) - every 2 hours")
print("\n📋 Timedelta Examples:")
print("  - timedelta(minutes=30): Every 30 minutes")
print("  - timedelta(hours=1): Every hour")
print("  - timedelta(hours=6): Every 6 hours")
print("  - timedelta(days=1): Daily")
print("  - timedelta(weeks=1): Weekly")
print("\n💡 Timedelta vs Cron:")
print("  - Timedelta: Đơn giản cho regular intervals")
print("  - Cron: Linh hoạt hơn cho complex patterns")


## 5. Catchup - Xử lý Missed Runs

Catchup quyết định có chạy các DAG runs đã bỏ lỡ từ start_date không.


In [ ]:
# DAG với Catchup = True
@dag(
    dag_id="catchup_true_example",
    schedule="@daily",
    start_date=pendulum.datetime(2024, 1, 1, tz="UTC"),  # Start từ 1/1/2024
    catchup=True,  # Sẽ chạy tất cả missed runs từ start_date
    tags=["scheduling", "catchup"],
)
def catchup_true_dag():
    """
    ### Catchup = True Example
    Nếu DAG được enable vào 1/10/2024, sẽ chạy tất cả runs từ 1/1 đến 1/10.
    """
    
    @task
    def catchup_task():
        """Task với catchup enabled"""
        print("Running catchup task")
        return "Catchup task completed"
    
    catchup_task()

# DAG với Catchup = False
@dag(
    dag_id="catchup_false_example",
    schedule="@daily",
    start_date=pendulum.datetime(2024, 1, 1, tz="UTC"),
    catchup=False,  # Chỉ chạy từ thời điểm hiện tại
    tags=["scheduling", "no-catchup"],
)
def catchup_false_dag():
    """
    ### Catchup = False Example
    Chỉ chạy từ thời điểm DAG được enable, không chạy missed runs.
    """
    
    @task
    def no_catchup_task():
        """Task với catchup disabled"""
        print("Running no-catchup task")
        return "No catchup task completed"
    
    no_catchup_task()

print("✅ Catchup Examples DAGs created!")
print("\n📊 Catchup Behavior:")
print("=" * 60)
print("Catchup = True:")
print("  - Chạy tất cả missed runs từ start_date")
print("  - Useful cho backfilling historical data")
print("  - Có thể tạo nhiều DAG runs cùng lúc")
print("\nCatchup = False:")
print("  - Chỉ chạy từ thời điểm hiện tại")
print("  - Recommended cho production DAGs")
print("  - Tránh tạo quá nhiều runs")
print("=" * 60)


## 6. Data Intervals - Hiểu Data Processing Windows

Data intervals xác định khoảng thời gian data được process trong mỗi DAG run.


In [ ]:
# DAG với Data Intervals
@dag(
    dag_id="data_intervals_example",
    schedule="@daily",
    start_date=pendulum.datetime(2024, 1, 1, tz="UTC"),
    catchup=False,
    tags=["scheduling", "data-intervals"],
)
def data_intervals_dag():
    """
    ### Data Intervals Example
    Hiểu cách data intervals hoạt động trong Airflow.
    """
    
    @task
    def process_data(**context):
        """Process data với data interval information"""
        # Data interval start và end
        data_interval_start = context['data_interval_start']
        data_interval_end = context['data_interval_end']
        
        # Logical date (execution date)
        logical_date = context['ds']
        
        print("=" * 60)
        print("Data Interval Information:")
        print(f"  Data Interval Start: {data_interval_start}")
        print(f"  Data Interval End: {data_interval_end}")
        print(f"  Logical Date (ds): {logical_date}")
        print(f"  Duration: {data_interval_end - data_interval_start}")
        print("=" * 60)
        
        # Ví dụ: Process data từ 2024-01-01 00:00 đến 2024-01-02 00:00
        print(f"\nProcessing data from {data_interval_start} to {data_interval_end}")
        
        return {
            "interval_start": data_interval_start.isoformat(),
            "interval_end": data_interval_end.isoformat(),
            "records_processed": 1000,
        }
    
    process_data()

# Create DAG
data_intervals_dag_instance = data_intervals_dag()

print("✅ Data Intervals DAG created!")
print("\n💡 Data Intervals:")
print("  - data_interval_start: Bắt đầu của data window")
print("  - data_interval_end: Kết thúc của data window")
print("  - logical_date (ds): Ngày logic của DAG run")
print("\n📊 Example với @daily schedule:")
print("  DAG run on 2024-01-02 processes data from:")
print("    Start: 2024-01-01 00:00:00")
print("    End:   2024-01-02 00:00:00")
print("    Logical Date: 2024-01-01")


## 7. Custom Timetables - Advanced Scheduling

Custom timetables cho phép tạo scheduling logic phức tạp không thể làm được với cron hoặc timedelta.


In [ ]:
# Custom Timetable: Chỉ chạy vào ngày làm việc (Monday-Friday)
from airflow.timetables.base import DagRunInfo, DataInterval, TimeRestriction, Timetable

class WorkdayTimetable(Timetable):
    """
    Custom timetable chỉ chạy vào ngày làm việc (Monday-Friday).
    """
    
    def infer_manual_data_interval(self, run_after: pendulum.DateTime) -> DataInterval:
        """Infer data interval cho manual runs"""
        # Chỉ chạy vào ngày làm việc
        while run_after.weekday() >= 5:  # Saturday=5, Sunday=6
            run_after = run_after.subtract(days=1)
        
        start = run_after.start_of("day")
        end = start.add(days=1)
        return DataInterval(start=start, end=end)
    
    def next_dagrun_info(
        self,
        last_automated_data_interval: DataInterval | None,
        restriction: TimeRestriction,
    ) -> DagRunInfo | None:
        """Calculate next DAG run"""
        if last_automated_data_interval is None:
            # First run
            start = restriction.earliest
        else:
            start = last_automated_data_interval.end
        
        # Tìm ngày làm việc tiếp theo
        while start.weekday() >= 5:  # Skip weekends
            start = start.add(days=1)
        
        end = start.add(days=1)
        
        # Check nếu vượt quá latest
        if restriction.latest and end > restriction.latest:
            return None
        
        return DagRunInfo.interval(start=start, end=end)

# DAG với Custom Timetable
@dag(
    dag_id="custom_timetable_example",
    schedule=WorkdayTimetable(),  # Custom timetable
    start_date=pendulum.datetime(2024, 1, 1, tz="UTC"),
    catchup=False,
    tags=["scheduling", "custom-timetable"],
)
def custom_timetable_dag():
    """
    ### Custom Timetable Example
    DAG chỉ chạy vào ngày làm việc (Monday-Friday).
    """
    
    @task
    def workday_task():
        """Task chỉ chạy vào ngày làm việc"""
        print("Running workday-only task")
        return "Workday task completed"
    
    workday_task()

# Create DAG
custom_timetable_dag_instance = custom_timetable_dag()

print("✅ Custom Timetable DAG created!")
print("\n💡 Custom Timetable:")
print("  - Inherit từ Timetable base class")
print("  - Implement next_dagrun_info() method")
print("  - Có thể tạo logic scheduling phức tạp")
print("\n📋 Use Cases:")
print("  - Business days only")
print("  - Skip holidays")
print("  - Custom business rules")
print("  - Event-driven scheduling")


## 8. Timezone Handling

Airflow sử dụng UTC làm timezone mặc định. Luôn sử dụng pendulum để xử lý timezone đúng cách.


In [ ]:
# DAG với Timezone Handling
@dag(
    dag_id="timezone_example",
    schedule="0 9 * * *",  # 9 AM UTC
    start_date=pendulum.datetime(2024, 1, 1, tz="UTC"),  # Luôn dùng UTC
    catchup=False,
    tags=["scheduling", "timezone"],
)
def timezone_dag():
    """
    ### Timezone Handling Example
    Luôn sử dụng UTC và pendulum cho timezone handling.
    """
    
    @task
    def timezone_task(**context):
        """Task với timezone handling"""
        # Airflow context luôn dùng UTC
        execution_date = context['data_interval_start']
        
        print("=" * 60)
        print("Timezone Information:")
        print(f"  Execution Date (UTC): {execution_date}")
        print(f"  Timezone: {execution_date.timezone_name}")
        
        # Convert sang timezone khác nếu cần
        vietnam_time = execution_date.in_timezone("Asia/Ho_Chi_Minh")
        print(f"  Vietnam Time: {vietnam_time}")
        print("=" * 60)
        
        return {
            "utc_time": execution_date.isoformat(),
            "vietnam_time": vietnam_time.isoformat(),
        }
    
    timezone_task()

# Create DAG
timezone_dag_instance = timezone_dag()

print("✅ Timezone DAG created!")
print("\n💡 Timezone Best Practices:")
print("  - Luôn sử dụng UTC cho start_date")
print("  - Sử dụng pendulum.datetime() với tz='UTC'")
print("  - Convert sang local timezone trong tasks nếu cần")
print("  - Không sử dụng datetime.datetime() (không có timezone)")
print("\n⚠️  Common Mistakes:")
print("  - Sử dụng datetime.datetime() thay vì pendulum")
print("  - Không specify timezone trong start_date")
print("  - Mix UTC và local timezone")


## 9. Schedule Examples - Real-world Scenarios

Các ví dụ scheduling patterns phổ biến trong thực tế.


In [ ]:
# Example 1: Business Hours Only (9 AM - 5 PM, Weekdays)
@dag(
    dag_id="business_hours_example",
    schedule="0 9-17 * * 1-5",  # 9 AM to 5 PM, Monday to Friday
    start_date=pendulum.datetime(2024, 1, 1, tz="UTC"),
    catchup=False,
    tags=["scheduling", "business-hours"],
)
def business_hours_dag():
    """Chạy trong giờ làm việc, ngày làm việc"""
    @task
    def business_task():
        print("Running during business hours")
        return "Business task completed"
    business_task()

# Example 2: End of Day Processing
@dag(
    dag_id="end_of_day_example",
    schedule="0 23 * * *",  # 11 PM daily
    start_date=pendulum.datetime(2024, 1, 1, tz="UTC"),
    catchup=False,
    tags=["scheduling", "eod"],
)
def end_of_day_dag():
    """Chạy vào cuối ngày để process daily data"""
    @task
    def eod_task():
        print("Running end-of-day processing")
        return "EOD task completed"
    eod_task()

# Example 3: High Frequency (Every 15 minutes)
@dag(
    dag_id="high_frequency_example",
    schedule="*/15 * * * *",  # Every 15 minutes
    start_date=pendulum.datetime(2024, 1, 1, tz="UTC"),
    catchup=False,
    tags=["scheduling", "high-frequency"],
)
def high_frequency_dag():
    """Chạy với tần suất cao cho real-time processing"""
    @task
    def frequent_task():
        print("Running high-frequency task")
        return "Frequent task completed"
    frequent_task()

# Example 4: Weekly Report (Monday morning)
@dag(
    dag_id="weekly_report_example",
    schedule="0 8 * * 1",  # 8 AM every Monday
    start_date=pendulum.datetime(2024, 1, 1, tz="UTC"),
    catchup=False,
    tags=["scheduling", "weekly"],
)
def weekly_report_dag():
    """Chạy hàng tuần để generate weekly reports"""
    @task
    def weekly_task():
        print("Generating weekly report")
        return "Weekly report generated"
    weekly_task()

print("✅ Real-world Scheduling Examples created!")
print("\n📋 Common Scheduling Patterns:")
print("=" * 60)
print("1. Business Hours:")
print("   Schedule: '0 9-17 * * 1-5'")
print("   Runs: 9 AM - 5 PM, Monday-Friday")
print("\n2. End of Day:")
print("   Schedule: '0 23 * * *'")
print("   Runs: 11 PM daily")
print("\n3. High Frequency:")
print("   Schedule: '*/15 * * * *'")
print("   Runs: Every 15 minutes")
print("\n4. Weekly Reports:")
print("   Schedule: '0 8 * * 1'")
print("   Runs: 8 AM every Monday")
print("=" * 60)


## 10. Best Practices và Common Pitfalls

### ✅ Best Practices:

1. **Start Date:**
   - Luôn sử dụng UTC timezone
   - Sử dụng `pendulum.datetime()` với `tz="UTC"`
   - Set start_date trong quá khứ để có thể backfill

2. **Catchup:**
   - Set `catchup=False` cho production DAGs
   - Chỉ dùng `catchup=True` cho backfilling
   - Monitor khi enable catchup để tránh quá tải

3. **Schedule Intervals:**
   - Sử dụng cron presets khi có thể (@daily, @hourly)
   - Custom cron cho complex patterns
   - Timedelta cho regular intervals

4. **Timezone:**
   - Luôn dùng UTC trong Airflow
   - Convert sang local timezone trong tasks nếu cần
   - Không mix timezones

5. **Data Intervals:**
   - Hiểu data_interval_start và data_interval_end
   - Sử dụng logical_date (ds) cho data partitioning
   - Đảm bảo data interval đúng với business logic

### ⚠️ Common Pitfalls:

1. **Timezone confusion**: Mix UTC và local timezone
2. **Catchup overload**: Enable catchup tạo quá nhiều runs
3. **Wrong start_date**: Set trong tương lai làm DAG không chạy
4. **Schedule mismatch**: Schedule không match với data availability
5. **Data interval misunderstanding**: Process sai data window


## 11. Tóm tắt và Next Steps

### ✅ Những gì đã học:
1. Cron expressions và presets (@daily, @hourly, etc.)
2. Custom cron patterns cho complex scheduling
3. Timedelta scheduling cho regular intervals
4. Catchup behavior và khi nào sử dụng
5. Data intervals và data processing windows
6. Custom timetables cho advanced scheduling
7. Timezone handling với UTC và pendulum
8. Real-world scheduling patterns
9. Best practices và common pitfalls

### 📚 Next Lab:
- **Lab 7**: End-to-End Pipeline Integration
- Tích hợp Airflow với Kafka, Spark, Databases
- Multi-service orchestration
- Error handling và recovery
- Monitoring và alerting

### 🔗 Useful Links:
- [Scheduling Documentation](https://airflow.apache.org/docs/apache-airflow/3.1.1/core-concepts/dags.html#scheduling)
- [Cron Expressions](https://airflow.apache.org/docs/apache-airflow/3.1.1/authoring-and-scheduling/cron.html)
- [Timetables](https://airflow.apache.org/docs/apache-airflow/3.1.1/authoring-and-scheduling/timetable.html)
- [Data Intervals](https://airflow.apache.org/docs/apache-airflow/3.1.1/core-concepts/dag-run.html#data-interval)

### 💡 Key Takeaways:

**Scheduling Options:**
- Cron presets: @daily, @hourly, etc. (đơn giản)
- Cron expressions: Custom patterns (linh hoạt)
- Timedelta: Regular intervals (đơn giản)
- Custom timetables: Complex logic (advanced)

**Important Concepts:**
- Data intervals: Khoảng thời gian data được process
- Catchup: Có chạy missed runs không
- Timezone: Luôn dùng UTC trong Airflow
- Start date: Ngày bắt đầu schedule

### 💡 Exercises:
1. Tạo DAG với schedule chạy mỗi 3 giờ
2. Tạo DAG chỉ chạy vào ngày làm việc
3. Implement custom timetable cho business logic
4. Xử lý timezone conversion trong tasks
5. Tạo DAG với catchup để backfill data
